# **라이브러리 정의**

In [1]:
# 뉴스 데이터 크롤링
import os, sys, datetime, time, json, urllib.request, urllib.parse

# **API KEY 설정**

In [2]:
# Naver Developer API Configuration
client_id = ''
client_secret = ''

# **함수 정의**

In [ ]:
def getRequestUrl(url):
    """ 1. GET Requests -> 2. GET Response """
    req = urllib.request.Request(url)

    # HTTP 헤더에 클라이언트 ID와 Secret 추가
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        # URL Open -> 응답값 반환
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print(f"[{datetime.datetime.now()}]Url Request Success")
            return response.read().decode('utf-8')
    # 예외 처리
    except Exception as e :
        print(e)
        print(f"[{datetime.datetime.now()}] Error for URL : {url}")
        return None

In [4]:
def getNaverSearch(node, srcText, start, display):
    """ 네이버 검색 API을 통하여 검색 후 검색 결과 반환 """

    # URL 파라미터값 정의
    base = "https://openapi.naver.com/v1/search"
    node = f"/{node}.json"
    parameters = f"?query={urllib.parse.quote(srcText)}&start={start}&display={display}"

    # 최종 URL 산출
    url = base + node + parameters

    # URL Request -> return: Response
    responseDecode = getRequestUrl(url)

    if(responseDecode == None):
        return None
    else:
        # return : json -> dictionary
        return json.loads(responseDecode)

In [ ]:
# 검색 결과에서 필요한 데이터를 추출하여 JSON 결과 리스트에 추가하는 함수 (Function to extract necessary data from search results and add to JSON result list)
def getPostData(post, jsonResult, cnt):
    """ 검색 결과 추출 -> JSON 결과 리스트에 추가 """
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    # link = post['link']

    # 발행일 전처리
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    # 추출된 데이터를 딕셔너리 형태로 jsonResult 리스트에 추가 (Add extracted data as a dictionary to jsonResult list)
    jsonResult.append(
        {
            'cnt':cnt,
            'title':title,
            'description':description,
            'org_link':org_link,
            'link':org_link,
            'pDate':pDate
        }
    )

In [ ]:
def main(input_text):
    # 뉴스 검색어 파라미터값 정의
    srcText = input_text    # 검색어
    node = 'news'           # API 검색 카테고리
    cnt = 0                 # 검색 데이터 수량
    jsonResult = []         # 검색 데이터가 담길 리스트

    # 최초 검색 사이클 실행 : 1 ~ 100개(max) 데이터 로드 **
    jsonResponse = getNaverSearch(node, srcText, 1, 100)

    # 전체 검색 결과 수량
    total = jsonResponse['total']

    # 무한 루프 : 검색한 뉴스 데이터를 모두 수집
    while((jsonResponse != None) and (jsonResponse['display'] != 0)):
        # 한 검색 사이클 -> 1 ~ 100개 데이터를 리스트에 append
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)

        # 다음 검색 사이클 실행 위치 정의
        start = jsonResponse['start'] + jsonResponse['display']

        # 다음 검색 사이클 실행
        jsonResponse = getNaverSearch(node, srcText, start, 100)

    print(f'전체 검색 : {total} 건') # 전체 검색 결과 수 출력 (Print total number of search results)

    # 결과를 JSON 파일로 저장 (Save results to a JSON file)
    with open(f'{srcText}_naver_{node}.json', 'w', encoding='utf8') as outfile:
        # JSON 데이터 Formatting
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys = True, ensure_ascii = False)
        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))